In [35]:
import json
import pandas as pd
import numpy as np
from collections import defaultdict, Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report, accuracy_score
from sklearn.naive_bayes import GaussianNB
from pprint import pprint
from string import punctuation
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from unidecode import unidecode
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier

In [36]:
class EP:
    
    def __init__(self, ethnicity):  
        
        assert ethnicity in ('iranian polish italian german japanese vietnamese' 
                            ' portuguese fijian indian croatian greek'
                             ' serbian spanish korean thai').split(), f'incorrect ethnicity: {ethnicity}!'
        
        datafile = f'/Users/ik/Data/names/training-{ethnicity}.csv'
        self.target_col = f'is_{ethnicity}'
        
        self.data = pd.read_csv(datafile).sample(frac=1.)
        self.data['full_name'] = self.data['full_name'].apply(lambda _: unidecode(_))
        
        assert self.target_col in self.data.columns, f'there is no {self.target_col} in data file!'  
        
        try:
            self.NODOUBT_FIRST_NAMES = list({line.strip() for line in open(f'/Users/ik/Data/names/real_{ethnicity}.txt','r').readlines() if line.strip()})
        except:
            self.NODOUBT_FIRST_NAMES = []
        
    def add_nodoubt_names(self):
        
        print(f'adding {len(self.NODOUBT_FIRST_NAMES)} first names..')
        
        self.data = pd.concat([self.data, pd.DataFrame({'full_name': self.NODOUBT_FIRST_NAMES, 
                                                        self.target_col: [1]*len(self.NODOUBT_FIRST_NAMES)})], 
                              ignore_index=True).sample(frac=1.)
        return self

In [37]:
class Selector(BaseEstimator, TransformerMixin):
    """
    select a columns from a data frame and return as a list
    """
    def __init__(self, col_name):
        self.col_name = col_name
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        return '_start_' + x[self.col_name] + '_end_'

class WordCount(BaseEstimator, TransformerMixin):
    """
    select a columns from a data frame and return as a list
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.apply(lambda _: len(_.split())).values.reshape(x.shape[0],1)
        return res

class NameLength(BaseEstimator, TransformerMixin):
    """
    return the length of the full name
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.str.len().values.reshape(x.shape[0],1)
        return res

class FirstLast(BaseEstimator, TransformerMixin):
    """
    is the first word longer than the last one
    """
    def __init__(self):
        pass
    
    def fit(self, x, y=None):
        return self

    def transform(self, x):
        res = x.apply(lambda _: np.argmax([len(p) for i, p in enumerate(_.split()) if i in [0,len(_.split())-1]])).values.reshape(x.shape[0],1)
        return res

In [38]:
if __name__ == '__main__':
    
    ic = EP('thai')
    #.add_nodoubt_names()
    
    # split into the training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(ic.data.drop([ic.target_col], axis=1), 
                                                        ic.data[ic.target_col],  test_size=0.25, 
                                                        random_state=391, stratify=ic.data[ic.target_col])
    

    pipe = Pipeline([('select_fullname', Selector('full_name')),  # df with 1 column
                        ('features', FeatureUnion(
                             [('char_level', CountVectorizer(strip_accents='ascii', analyzer='char', 
                                                          ngram_range=(1, 4))),
                              ('word_level', CountVectorizer(strip_accents='ascii', analyzer='word',
                                                       ngram_range=(1,2))),
                              ('word_count', WordCount()),
                              ('full_name_length', NameLength()),
                              ('firstlast', FirstLast())],
                                 transformer_weights={
                                            'char_level': 0.8,
                                            'word_level': 0.5,
                                            'word_lengths': 1.0,
                                                        })
                     ),
                     ('normalise', Normalizer()),
#                      ('pca', TruncatedSVD(n_components=120)),
                     ('clf', VotingClassifier(estimators=[('sgd', SGDClassifier(max_iter=1000)), 
                                                           ('rf', RandomForestClassifier()),
                                                            ('svc', SVC(C=0.0001)),
                                                             ('abst', AdaBoostClassifier(n_estimators=100)),
                                                             ('gb', GradientBoostingClassifier())], voting='hard'))
#                     ('clf', SGDClassifier(max_iter=1000))
                    ])
    
    param_grid = {
#         'features__transformer_weights': [{'char_level': 0.2,
#                                                      'word_level': 0.1,
#                                                      'word_count':0.4,
#                                                     'full_name_length': 0.5}, 
#                                                    {'char_level': 0.9,
#                                                      'word_level': 0.2,
#                                                      'word_lengths':0.9,
#                                                    'full_name_length': 0.3},
#                                                    {'char_level': 0.9,
#                                                      'word_level': 0.3,
#                                                      'word_count':0.2,
#                                                    'full_name_length': 0.7}],
                    "clf__sgd__loss": ['modified_huber'], # also 'hinge', 'perceptron', 'log'
                     'clf__sgd__penalty': ['l2'],  # 'elasticnet', 'l1'
                  'clf__rf__n_estimators': [100, 200],
                  'clf__rf__max_depth': [None, 3],
                    'clf__svc__C': [1e-5, 1e-4, 1e-2, 1]}  # default is 1.
#                  'pca__n_components': [30, 60, 120, 180]}
    
    grid_search = GridSearchCV(pipe, param_grid=param_grid, verbose=10)
    
    grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-05 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    6.6s remaining:    0.0s


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-05, score=0.9364161849710982, total=   5.8s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-05 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   13.5s remaining:    0.0s


[CV]  clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-05, score=0.9595375722543352, total=   6.1s
[CV] clf__rf__max_depth=None, clf__rf__n_estimators=100, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1e-05 


...

[CV]  clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1, score=0.9219653179190751, total=   6.8s
[CV] clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


[CV]  clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1, score=0.9494219653179191, total=   6.1s
[CV] clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1 


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
[Parallel(n_jobs=1)]: Done  48 out of  48 | elapsed:  5.8min finished


[CV]  clf__rf__max_depth=3, clf__rf__n_estimators=200, clf__sgd__loss=modified_huber, clf__sgd__penalty=l2, clf__svc__C=1, score=0.9334298118668596, total=   6.2s


In [39]:
print(classification_report(y_test, grid_search.predict(X_test)))

             precision    recall  f1-score   support

          0       0.93      0.98      0.96       450
          1       0.96      0.86      0.91       242

avg / total       0.94      0.94      0.94       692



/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [40]:
test_data = pd.DataFrame({'full_name': ['han solo', 'xi','babak', 'igor souza', 'yash patel', 'david perron',
                                        'andrzej woods','otto floms', 'suzuki', 'toyota', 
                                        'edilson', 'kawasaki','silva', 'james tran', 
                                        'bo nguyen', 'zlatan ibrahimovic', 'ronaldo barbosa', 'ahmad lopes zeto', 
                                        'bob', 'mario razzi', 'john reed', 'andreas vlachos', 'con', 'nemanja vidic']})

test_data['prediction'] = grid_search.predict(test_data)
test_data['prediction'] = test_data['prediction'].apply(lambda _: 'yes' if _ else 'no')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [41]:
test_data

,full_name,prediction
0,han solo,no
1,xi,no
2,babak,no
3,igor souza,no
4,yash patel,no
5,david perron,no
6,andrzej woods,no
7,otto floms,no
8,suzuki,no
9,toyota,no


In [42]:
grid_search.best_params_

{'clf__rf__max_depth': None,
 'clf__rf__n_estimators': 100,
 'clf__sgd__loss': 'modified_huber',
 'clf__sgd__penalty': 'l2',
 'clf__svc__C': 0.0001}

In [43]:
for t in zip(X_test.values, grid_search.predict(X_test), y_test.values):
    if t[1] != t[2]:
        print(f'{t[0][0].upper()} predicted {t[1]} but it is {t[2]}')

WITITA BALEE predicted 0 but it is 1
THIRAYU BANHAN predicted 0 but it is 1
PRIYADARSHINI CHATTERJEE predicted 1 but it is 0
CHARIN BOODHAD predicted 0 but it is 1
IRFAN DOLAH predicted 0 but it is 1
SORAYA LATEH predicted 0 but it is 1
SUREE SUKHA predicted 0 but it is 1
SHAKTI CHATTOPADHYAY predicted 1 but it is 0
FARID MADSOH predicted 0 but it is 1
MARCO BALLINI predicted 0 but it is 1
ADISAK RUMKAE predicted 0 but it is 1
KORLAB TANTRATANAI predicted 0 but it is 1
ANURAK SRIKERD predicted 0 but it is 1
JANIE TIENPHOSUWAN predicted 0 but it is 1
SUTEE NOIPUK predicted 0 but it is 1
MAYLADA SUSRI predicted 0 but it is 1
WANNA BUAKAEW predicted 0 but it is 1
SADHANA SARGAM predicted 1 but it is 0
ROSENAN KANOH predicted 0 but it is 1
ANAN BUASANG predicted 0 but it is 1
SUBRAHMANYAN CHANDRASEKHAR predicted 1 but it is 0
JARUNEE SANNOK predicted 0 but it is 1
RUJIRA UKASRI predicted 0 but it is 1
APIWICH PHULEK predicted 0 but it is 1
ANAWIN JUJEEN predicted 0 but it is 1
ALISA SENGSA

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [44]:
grid_search.best_estimator_

Pipeline(memory=None,
     steps=[('select_fullname', Selector(col_name='full_name')), ('features', FeatureUnion(n_jobs=1,
       transformer_list=[('char_level', CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True...      warm_start=False))],
         flatten_transform=None, n_jobs=1, voting='hard', weights=None))])

In [45]:
X_test

,full_name
2251,phimonrat phisarayabud
87,papharat khotchum
482,nguyen thi kim lien
1518,noppanon kachaplayuk
770,maria bernal
2754,wanida kotruang
1943,davide caiti
1911,kittipong phuthawchueak
2145,samuele guddo
479,branko tucak
